# Script Set-Up

## Import Packages

In [ ]:
%matplotlib qt

In [ ]:
import os
import numpy as np
import mne
from matplotlib import pyplot as plt
import pandas as pd
import matplotlib.colors as mcolors
import math
import numpy.ma as ma
import statsmodels.formula.api as sm
import scipy

## Select Correct Working Directory

In [ ]:
os.chdir('../../') 

## Define Participant List 
- Selects data we want to load 

In [ ]:
ts_args = dict(gfp=True, time_unit='ms') # for styling plots for comparing evokeds (?)
topomap_args = dict(sensors=False, time_unit='ms') # for styling topomaps (?)

ID_list = ['P1', 'P3', 'P4', 'P5', 'P7', 'P8', 'P10', 'P11', 'P12', 'P13', 'P14', 'P15', 'P16',
           'P17', 'P19', 'P20', 'P21', 'P22', 'P23', 'P24', 'P25', 'P26', 'P27', 'P28', 'P29', 'P30'] 

number_of_blocks = {'P1':20, 'P3':16, 'P4':20, 'P5':20, 'P7':20, 'P8':20, 'P10':20, 'P11':16,
                    'P12':18,  'P13':16, 'P14':16, 'P15':17, 'P16':16, 'P17':17, 'P19':16, 
                    'P20':18, "P21": 20, "P22": 18, "P23": 20, "P24":18, "P25": 20, "P26": 20, 
                    'P27': 20, 'P28': 20, "P29": 20, "P30": 20}

index = ID_list.index('P1') # select participant here
ID = ID_list[index]
num_blocks = number_of_blocks[ID]

## Read in Raw Data
- Loads raw time-series data

### Read first block

In [ ]:
i=1 # sets the block you want to load in
raw_arrays = []
fname = 'Harvey EEG Data/' + str(ID) + '/' + str(ID) + '_' + str(i) + '.bdf'
raw = mne.io.read_raw_bdf(fname, preload = True) # loads in raw file
raw_arrays.append(raw.get_data())

### Read in mutiple files from one participant


In [ ]:
for i in range (2, (num_blocks + 1)): # sets the rest of the blocks you want to load in
    fname = 'Harvey EEG Data/' + str(ID) +'/'+ str(ID) + '_' + str(i) + '.bdf'
    single_raw = mne.io.read_raw_bdf(fname, preload = True)
    raw_arrays.append(single_raw.get_data())
    raw = mne.io.concatenate_raws([raw, single_raw],verbose=False) # concatenates files together

# Detrend the Data to Remove Slow-Drifts

1. Get raw data in array form (n_channels, n_times) 
2. Apply linear detrend to each channel using scipy.signal.detrend
3. Put data back into mne object form

## Apply detrending function 

In [ ]:
detrended_raw_arrays = []

import scipy.signal
    
for i in range(0, len(raw_arrays)):
    data = raw_arrays[i]
    detrended_raw_array = np.zeros(data.shape)
        
    for j in range(0, raw_arrays[0].shape[0]): # iterate through all channels
        detrended_raw_array[j] = scipy.signal.detrend(data[j], axis = 0, type = "linear", bp = 0)
        
    detrended_raw_arrays.append(detrended_raw_array)

## Put data back in MNE object form

In [ ]:
detrended_raw_obj = []
info = raw.info

for i in range(0, len(detrended_raw_arrays)):
    detrended_raw_obj.append(mne.io.RawArray(detrended_raw_arrays[i], info = info))

# Low Pass Filter

In [ ]:
h_freq = 30 
for i in range(0, len(detrended_raw_obj)):
    detrended_raw_obj[i].filter(h_freq = h_freq, l_freq = None)

# Concatenate Blocks

In [ ]:
raw_for_events = raw
raw =  mne.io.concatenate_raws(detrended_raw_obj, verbose=False)

# Redefine Channels
- Channels 1-128 are defined as EEG (scalp channels)

- Redefiining channel 129 and 130 as ECG channels

- Redefining channels 131-144 as misc

In [ ]:
## Defining channels 129 and 130 as EOG channels ##
raw.set_channel_types({raw.ch_names[128]: 'ecg'}) 
raw.set_channel_types({raw.ch_names[129]: 'ecg'})

## Defining channels 131-144 as miscellaneous ##
for n in range(130,143):
    raw.set_channel_types({raw.ch_names[n]: 'misc'})

# Create Bipolar EOG Channel
- For removing blink artefacts 

In [ ]:
anode = 'EXG1' # computes anode minus cathode, so upper = anode, lower = cathode
cathode = 'EXG2'
ch_name = 'Bipolar_EOG' # setting name for resulting bipolar channel
ch_info = None # can add dict with info about new channel
drop_refs = False # if True, single external channels are dropped from data, False retains them
copy = False # False modifies in place, True creates a copy and acts on that

mne.set_bipolar_reference(raw, anode = anode, cathode = cathode, ch_name = ch_name, ch_info = ch_info, 
                          drop_refs = drop_refs, copy = copy)

raw.set_channel_types({'Bipolar_EOG': 'eog'}) # Setting Bipolar_Eog as eog type

# Define Montage
Defining the montage as the 128 Biosemi

In [ ]:
raw.set_montage('biosemi128')

# Interpolate Bad Channels
- Lists the channels selected for interpolation for each participant


In [ ]:
%matplotlib inline

if ID == 'P1':
    raw.info['bads'] = [ 'C25', 'B28', 'C14', 'A20', 'A9', "C16", "B29"] 
    ch_names_to_skip = []
elif ID == 'P3':
    raw.info['bads'] = [ 'C25', 'A8', 'C3']
    ch_names_to_skip = []
elif ID == 'P4':
    raw.info['bads'] = [ 'C25']
    ch_names_to_skip = []
elif ID == 'P5': # not included
    raw.info['bads'] = ['C14', 'C25', 'A9', 'A16', 'B8', 'A10', 'A17', 'B10', 'B11', 'B7', 'A29']
elif ID == 'P7':
    raw.info['bads'] = [ 'C25', 'C29']
    ch_names_to_skip = []
elif ID == 'P8':
    raw.info['bads'] = [ 'C14', 'C25']
    ch_names_to_skip = []
elif ID == 'P10':
    raw.info['bads'] = [ 'C14', 'C25', 'A13', 'D4', 'D27']
    ch_names_to_skip = ['B26', 'B14']
elif ID == 'P11':
    raw.info['bads'] = [ 'C14', 'C25']
    ch_names_to_skip = []
elif ID == 'P12':
    raw.info['bads'] = [ 'C25', 'D9']
    ch_names_to_skip = []
elif ID == 'P13':
    raw.info['bads'] = [ 'C25']
    ch_names_to_skip = []
elif ID == 'P14':
    raw.info['bads'] = [ 'C25', 'C17'] # maybe not C17
    ch_names_to_skip = []
elif ID == 'P15':
    raw.info['bads'] = [ 'C25', 'D26']
    ch_names_to_skip = []
elif ID == 'P16':
    raw.info['bads'] = [ 'C25', 'D6', 'A12', 'B8', 'B6', 'B5']
    ch_names_to_skip = []
elif ID == 'P17':
    raw.info['bads'] = ['C5', 'D4']
    ch_names_to_skip = []
elif ID == 'P19':
    raw.info['bads'] = [ 'C25', 'B27', 'B28', 'C9', 'D8', 'D9', 'A11', 'B4', 'D4']
    ch_names_to_skip = []
elif ID == 'P20':
    raw.info['bads'] = [ 'C25', 'B20', 'B2', 'D5', 'C30']
    ch_names_to_skip = []
elif ID == 'P21':
    raw.info['bads'] = ['B7']
    ch_names_to_skip = []
elif ID == 'P22':
    raw.info['bads'] = ['A32', 'C6', 'C7', 'D5', 'C8', 'D6', 'A9']
    ch_names_to_skip = []
elif ID == 'P23':
    raw.info['bads'] = ['D11', 'C1', 'D7']
    ch_names_to_skip = []
elif ID == 'P24':
    raw.info['bads'] = ['C3', 'C2', 'B29', 'C26', 'C21', 'C6', 'C7', 'B27']
    ch_names_to_skip = []
elif ID == 'P25':
    raw.info['bads'] = ['D20', 'C22', 'D10', 'C3', 'B29', 'B3', 'B4', 'B32', 'B25']
    ch_names_to_skip = []
elif ID == 'P26':
    raw.info['bads'] = ['A13']
    ch_names_to_skip = []
elif ID == 'P27':
    raw.info['bads'] = []
    ch_names_to_skip = []
elif ID == 'P28':
    raw.info['bads'] = ["A32", "B22", "B29"]
    ch_names_to_skip = []
elif ID == 'P29':
    raw.info['bads'] = ["D10"]
    ch_names_to_skip = []
elif ID == 'P30':
    raw.info['bads'] = []
    ch_names_to_skip = []
    
raw.info['bads']
raw.interpolate_bads()

# Apply Average Reference
- Re-referencing to the average of the 128 scalp electrodes

In [ ]:
raw.set_eeg_reference('average', projection = False)

# Find Events
- Finding all the events on the Status channel. 

- mne.find_events returns a n_events by 3 array describing the events 

    - column 1 indicates event time in samples
    - column 2 indicates value of status channel immediately before the event
    - column 3 indicates event ID 

In [ ]:
stim_channel = 'Status' # Name of the status channel that tracks the triggers
min_duration = 0.002 # 0 is MNE default # 0.002 # specifies the minimum duration of a change (in seconds) in stim channel that is required for it to be consideredan event
initial_event = True # if True an event is created if the stim channel has a value different from 0 as its first sample

events_detrended = mne.find_events(raw_for_events, stim_channel = stim_channel, min_duration = min_duration, initial_event = initial_event)
raw.add_events(events_detrended, stim_channel = stim_channel, replace = True)
min_duration = 0 # have to change this 
events = mne.find_events(raw, stim_channel = stim_channel, min_duration = min_duration, initial_event = initial_event)

# Track Trial Numbers
- This is only applicable to a small number of participants (P7) who have certain triggers missing on some trials

- Need to know which trials don't have an encoding, end encode, reproduction or response trigger in them

- So this cell marks down what the trigger values are in each trial by iterating through the events array, first checking if the trigger is an encode trigger, and then checks if the other triggers that should be there are indeed there

- Trigger values from trials that do not contain all events are then removed from the events, so that only trials with all 4 encoding, end encode, repo and response triggers are included for the analyses

In [ ]:
num_trials = 45 * num_blocks # 5 mini blocks of 9 trials in each block = 45 trials

trials = pd.DataFrame({"Trial_Number": np.arange(0,num_trials), 
                       "Encode": np.zeros(num_trials),
                       "Encode_n": np.zeros(num_trials),
                       "SI": np.zeros(num_trials),
                       "End_Encode": np.zeros(num_trials),
                       "End_Encode_n": np.zeros(num_trials),
                       "Repo": np.zeros(num_trials),
                       "Repo_n": np.zeros(num_trials),
                       "Response": np.zeros(num_trials),
                        "Response_n": np.zeros(num_trials),
                        "Gap_Time": np.zeros(num_trials),
                        "Response_Time": np.zeros(num_trials), 
                        "Encode_Time": np.zeros(num_trials)})

## Storing Trigger Value and Location in Events Array on Each Trial (trig = 0 if no trigger on that trial) ##

encode_triggers = [51, 52, 53, 54, 55, 56, 57, 58, 59]
end_triggers = [101, 102, 103, 104, 105, 106, 107, 108, 109]
repo_triggers = [151, 152, 153, 154, 155, 156, 157, 158, 159]
response_locked_triggers = [201, 202, 203, 204, 205, 206, 207, 208, 209]

index = 0
for n in range(0, len(events) - 3): 
    set_index = 0
    if events[n,2] in encode_triggers: # first checks if the trigger is an encode trigger 
        trials.at[index, "Encode"] = events[n,2] # marks the encode trigger value
        trials.at[index, "Encode_n"] = n # marks the row value in the events array
        if events[n,2] == 51: # adding 05/12/22
            trials.at[index, "SI"] = 600
        elif events[n, 2] == 52:
            trials.at[index, "SI"] = 650
        elif events[n, 2] == 53:
            trials.at[index, "SI"] = 700
        elif events[n, 2] == 54:
            trials.at[index, "SI"] = 750
        elif events[n, 2] == 55:
            trials.at[index, "SI"] = 800
        elif events[n, 2] == 56:
            trials.at[index, "SI"] = 850
        elif events[n, 2] == 57:
            trials.at[index, "SI"] = 900
        elif events[n, 2] == 58:
            trials.at[index, "SI"] = 950
        elif events[n, 2] == 59:
            trials.at[index, "SI"] = 1000
        
        if events[n + 1, 2] in end_triggers:
            trials.at[index, "End_Encode"] = events[n + 1, 2] # marks the trigger value
            trials.at[index, "End_Encode_n"] = n + 1 # marks the row value in the events array
            encode_samps = events[n + 1, 0] - events[n, 0] # 5/12/22
            encode_ms = encode_samps / 512 * 1000 # converting to ms
            trials.at[index, "Encode_Time"] = encode_ms
            if events[n + 2, 2] in repo_triggers: # adding gap time to dataframe
                gap_samps = events[n + 2, 0] - events[n + 1, 0] # repo trigger - end trigger
                gap_ms = gap_samps / 512 * 1000 # converting to ms
                trials.at[index, "Gap_Time"] = gap_ms
        elif events[n + 1, 2] not in end_triggers:
            trials.at[index, "End_Encode"] = 0 
            trials.at[index, "End_Encode_n"] = 0
            
        if events[n + 2, 2] in repo_triggers:
            trials.at[index, "Repo"] = events[n + 2, 2] # marks the trigger value
            trials.at[index, "Repo_n"] = n + 2
        elif events[n + 2, 2] not in repo_triggers:
            trials.at[index, "Repo"] = 0
            trials.at[index, "Repo_n"] = 0
            
        if events[n + 3, 2] in response_locked_triggers:
            trials.at[index, "Response"] = events[n + 3, 2]
            trials.at[index, "Response_n"] = n + 3
            rt_samps = events[n + 3, 0] - events[n + 2, 0]
            rt = rt_samps / 512 * 1000
            trials.at[index, "Response_Time"] = rt
        elif events[n + 3, 2] not in response_locked_triggers:
            trials.at[index, "Response"] = 0
            trials.at[index, "Response_n"] = 0
        index += 1
        
    elif events[n, 2] not in encode_triggers: # if the trigger is not an encode trigger, it checks the subsequent triggers to see if the other 3 line up but the encode is missing
        trials.at[index, "Encode"] = 0
        trials.at[index, "Encode_n"] = 0
        if events[n + 1, 2] in end_triggers:
            trials.at[index, "End_Encode"] = events[n + 1, 2]
            trials.at[index, "End_Encode_n"] = n + 1
            set_index = 1
        if events[n + 2, 2] in repo_triggers:
            trials.at[index, "Repo"] = events[n + 2, 2]
            trials.at[index, "Repo_n"] = n + 2
            set_index = 1
        if events[n + 3, 2] in response_locked_triggers:
            trials.at[index, "Response"] = events[n + 3, 2]
            trials.at[index, "Response_n"] = n + 3
            set_index = 1
        if set_index == 1:
            index += 1

# Identify Trigger Values and Trial Numbers of Trials With Missing Triggers

In [ ]:
## Store the Row Value of the Valid Triggers in the Events Array For Trials in Which another trigger is missing #
## This means these triggers can be removed from the Events array so that only trials with all 4 triggers remain 

trials_with_no_encode_triggers = []
trials_with_no_gap_triggers = []
trials_with_no_repo_triggers = []
trials_with_no_response_triggers = []

rows_to_delete = []

for i in range(0, len(trials)):
    if trials.at[i, "Encode"] == 0: # Trials with no Encode
        if i not in rows_to_delete:
            rows_to_delete.append(i)
        if trials.at[i, "End_Encode"] != 0:
            trials_with_no_encode_triggers.append(trials.at[i, "End_Encode_n"])
        if trials.at[i, "Repo"] != 0:
            trials_with_no_encode_triggers.append(trials.at[i, "Repo_n"])
        if trials.at[i, "Response"] != 0:
            trials_with_no_encode_triggers.append(trials.at[i, "Response_n"])
            
    if trials.at[i, "End_Encode"] == 0: ## Trials with no gap
        if i not in rows_to_delete:
            rows_to_delete.append(i)
        if trials.at[i, "Encode"] != 0:
            trials_with_no_gap_triggers.append(trials.at[i, "Encode_n"])
        if trials.at[i, "Repo"] != 0:
            trials_with_no_gap_triggers.append(trials.at[i, "Repo_n"])
        if trials.at[i, "Response"] != 0:
            trials_with_no_gap_triggers.append(trials.at[i, "Response_n"])
            
    if trials.at[i, "Repo"] == 0: ## Trials with no Repo
        if i not in rows_to_delete:
            rows_to_delete.append(i)
        if trials.at[i, "Encode"] != 0:
            trials_with_no_repo_triggers.append(trials.at[i, "Encode_n"])
        if trials.at[i, "End_Encode"] != 0:
            trials_with_no_repo_triggers.append(trials.at[i, "End_Encode_n"])
        if trials.at[i, "Response"] != 0:
            trials_with_no_repo_triggers.append(trials.at[i, "Response_n"])
        
    if trials.at[i, "Response"] == 0: ## Trials with no Response
        if i not in rows_to_delete:
            rows_to_delete.append(i)
        if trials.at[i, "Encode"] != 0:
            trials_with_no_response_triggers.append(trials.at[i, "Encode_n"])
        if trials.at[i, "End_Encode"] != 0:
            trials_with_no_response_triggers.append(trials.at[i, "End_Encode_n"])
        if trials.at[i, "Repo"] != 0:
            trials_with_no_response_triggers.append(trials.at[i, "Repo_n"])

# Delete Missing Events 
- The below cell removes any events that are in trials that do not contain all 4 events, e.g removes encoding triggers that have no corresponding end encode, repo or response triggers

In [ ]:
triggers_for_deletion = np.unique(np.array(trials_with_no_encode_triggers + trials_with_no_gap_triggers + trials_with_no_repo_triggers + trials_with_no_response_triggers, dtype = "int"))

if len(triggers_for_deletion) > 0:
    events = np.delete(arr = events, obj = triggers_for_deletion, axis = 0)

In [ ]:
trials.drop(rows_to_delete, inplace = True)

# Perform Test Behavioural Analysis

In [ ]:
from matplotlib import pyplot as plt

av_response_time = np.zeros(9) # output array for mean rt for each sample interval condition
median_response_time = np.zeros(9) # # output array for median rt for each sample interval condition

fig, axs = plt.subplots(1,9,figsize=(15,4),sharey=True) # creating axes for big plot below

sample_interval_durations = np.arange(600,1001,50) # (600, 650, 700, 750, 800, 850, 900, 950, 1000)

for i in range(0, len(sample_interval_durations)):
    av_response_time[i] = trials["Response_Time"][trials["Encode"] == encode_triggers[i]].mean()
    median_response_time[i] = trials["Response_Time"][trials["Encode"] == encode_triggers[i]].median()
    axs[i].hist(trials["Response_Time"][trials["Encode"] == encode_triggers[i]])
    axs[i].axvline(np.average(trials["Response_Time"][trials["Encode"] == encode_triggers[i]]),color='k',linewidth=4, alpha = 0.7) # vertical line for mean
    axs[i].axvline(np.median(trials["Response_Time"][trials["Encode"] == encode_triggers[i]]),color='g',linewidth=4, alpha = 0.7) # vertical line for median
    axs[i].set_title(sample_interval_durations[i]) # sets title name for individual histogram
    axs[i].set_xlim([500,1500])
    axs[i].set_xlabel('Response \n Times (ms)')

In [ ]:
from scipy import stats
slope, intercept, r_value, p_value, std_err = stats.linregress(trials["SI"], trials["Response_Time"])

## Plotting ALL Sample Interval/Encoding Time Versus ALL Response Times ##

plt.subplots(1,1,figsize=(5,5))

plt.plot(trials["SI"], trials["Response_Time"],'.', alpha = 0.2) # plots response time in each trial against sample interval in each trial
plt.plot(sample_interval_durations, av_response_time,':ob', label = 'Average Reproduction', alpha = 1) # plots average response time for each condition
plt.plot(sample_interval_durations, median_response_time,':og', label = 'Median Reproduction', alpha = 1) # plots median response time for each condition 

plt.plot(sample_interval_durations,sample_interval_durations,'-.k',label = 'Veridical Encoder') # Plots what a veridical processor would reproduce (m = 1)
plt.plot(sample_interval_durations,800*np.ones(9),'--k',label = 'Average Encoder') # Plots what a perfectly average processor would reproduce (m = 0)
plt.xlabel('Sample Interval (ms)')
plt.ylabel('Response Time (ms)')
plt_title = str(ID) + ' Response Times'

plt.title(plt_title)
plt.ylim((0,1500))
#plt.legend()
if p_value <0.05:
    plt.text(850,300,r'$Slope$='+ str(np.round(slope,2)) + r'$^*$')
else:
    plt.text(850,300,r'$Slope$='+ str(np.round(slope,2)) )

plt.text(850,200,r'$Intercept$='+ str(np.round(intercept,2)))
plt.text(850,100,r'$R^2$=' + str(np.round(r_value**2,4)))

# Define Event Dictionary

## Encoding Events
|Trigger|| Event|
|-------||------|
| 51    || Encode 600ms|
| 52    || Encode 650ms|
| 53    || Encode 700ms|
| 54    || Encode 750ms|
| 55    || Encode 800ms|
| 56    || Encode 850ms|
| 57    || Encode 900ms|
| 58    || Encode 950ms|
| 59    || Encode 1000ms|



## End of Encoding Events
|Trigger|| Event|
|-------||------|
| 101    || End Encode 600ms|
| 102    || End Encode 650ms|
| 103    || End Encode 700ms|
| 104    || End Encode 750ms|
| 105    || End Encode 800ms|
| 106    || End Encode 850ms|
| 107    || End Encode 900ms|
| 108    || End Encode 950ms|
| 109    || End Encode 1000ms|



## Start of Reproduction 
|Trigger|| Event|
|-------||------|
| 151    || Start 600ms|
| 152    || Start 650ms|
| 153    || Start 700ms|
| 154    || Start 750ms|
| 155    || Start 800ms|
| 156    || Start 850ms|
| 157    || Start 900ms|
| 158    || Start 950ms|
| 159    || Start 1000ms|


## Response
|Trigger|| Event|
|-------||------|
| 201    || Response 600ms|
| 202    || Response 650ms|
| 203    || Response 700ms|
| 204    || Response 750ms|
| 205    || Response 800ms|
| 206    || Response 850ms|
| 207    || Response 900ms|
| 208    || Response 950ms|
| 209    || Response 1000ms|

# Epoch Data into 3 Segments for Analysis
- Encoding epoch = pre-interval to post-interval

- Reproduction epoch = pre-cue (pre start of reproduction phase) - post-cue

- Response-Locked epoch = pre-response - post response

## Select Baseline Period
- Can select to baseline each epoch to its own pre-epoch period (baseline = 0)

- Can select to baseline each epoch to the pre-encoding phase interval (baseline = 1)

In [ ]:
baseline_setting = 1

if baseline_setting == 1:
    baseline_name = "pre_interval_baseline"
else:
    baseline_name = "own_baseline"

## Create Encoding Epochs
- Epochs are 1200ms duration, locked to the start of the encoding epoch (indicated by sample interval onset), using 200ms pre-stimulus period as baseline

In [ ]:
encode_event_dict = {'Encode/600':51,'Encode/650':52,'Encode/700':53,'Encode/750':54,'Encode/800':55,
                     'Encode/850':56,'Encode/900':57,'Encode/950':58,'Encode/1000':59}

encode_epochs = mne.Epochs(raw, events, event_id = encode_event_dict, preload = True)#, tmin=-0.1, 
                    #tmax=1.1,reject=reject_criteria, preload=True,baseline=(-.1,0),verbose=False);
  
reject_criteria = dict(eeg = 100e-6, eog = 200e-6)  # 100 μV for scalp, # 200 μV for bipolar,

# Seeing how many epochs in each condition
original_num_epochs = np.zeros(9)

for i in range(0, 9):
    name = "Encode/" + str(sample_interval_durations[i])
    encode = encode_epochs[name]
    original_num_epochs[i] = len(encode)

### Add RT related Metadata to the Encoding Epochs

In [ ]:
rt=events[encode_epochs.selection+3]-events[encode_epochs.selection+2]
rt=rt[:,0]*1000/512
mean_rt=np.zeros(len(rt))
median_rt=np.zeros(len(rt))

gaps = events[encode_epochs.selection+2]-events[encode_epochs.selection+1]
gaps=gaps[:,0]*1000/512

RT_gap = rt + gaps

for j in range(0,len(av_response_time)):
    mean_rt[encode_epochs.events[:,2]==51+j]=av_response_time[j]+50
    median_rt[encode_epochs.events[:,2]==51+j]=median_response_time[j]+50

Over_Under=np.zeros(len(rt))
Over_Under[rt>median_rt]=1
Trigger=encode_epochs.events[:,2]
encoding_metadata={'trial_number':range(len(encode_epochs.events)),
         'Response_Time':rt,
         'Over_Under':Over_Under,
         'Mean_RT':mean_rt,
         'Median_RT':median_rt,
         'Trigger':Trigger,
         'Gap': gaps,
         'Reject_Artif': np.zeros(len(encode_epochs.events)), 
         'RT_gap': RT_gap}
encoding_metadata=pd.DataFrame(encoding_metadata)

### Create Encoding Epochs Object

In [ ]:
encoding_epoch_events = encode_epochs.events 
event_id = encode_event_dict
preload = False
verbose = False
metadata = encoding_metadata 
detrend = None # 

if baseline_setting == 0:
    tmin = -0.2 
    tmax = 1.2 
    reject = None
    baseline = (-0.2, 0) # should apply baseline, from mne: "Correction is applied to each epoch and channel individually in the following way: Calculate the mean signal of the baseline period. Subtract this mean from the entire epoch."

elif baseline_setting == 1:
    tmin = -0.2
    tmax = 1.2 
    reject = None
    baseline = (-0.2, 0) 

encode_epochs = mne.Epochs(raw, encoding_epoch_events, event_id = encode_event_dict, tmin = tmin, tmax = tmax, 
                           reject = reject, preload = preload, detrend = detrend, baseline = baseline, 
                           verbose = verbose, metadata = metadata)

encode_epochs.load_data()
encode_epochs_copy = encode_epochs.copy()

## Create Baseline Epochs 
- Same as above but no baseline correction applied and only includes the baseline period

In [ ]:
encoding_epoch_events = encode_epochs.events 
event_id = encode_event_dict
preload = False
verbose = False 
metadata = encoding_metadata 
detrend = None #1 is linear detrend

tmin = tmin
tmax = 0
reject = None
baseline = None # not applying baseline correction so I can take out baseline period

baseline_epochs = mne.Epochs(raw, encoding_epoch_events, event_id = encode_event_dict, tmin = tmin, tmax = tmax, 
                           reject = reject, preload = preload, detrend = detrend, baseline = baseline, 
                           verbose = verbose, metadata = metadata)
baseline_epochs.load_data()

### Average Across All Time Points in the Baseline Period

In [ ]:
baseline_epochs_data = baseline_epochs.get_data()
baseline_period = np.mean(baseline_epochs_data, axis = 2)

## Apply Rejection Criteria To Encoding Epochs

In [ ]:
encode_epochs_data = encode_epochs.get_data()
n_epochs, n_channels, n_times = encode_epochs_data.shape
rejected_epochs = []

ch_names = raw.ch_names
ch_dict = {}

for ch in ch_names[0:128]:
    ch_dict[ch] = 0

ch_dict[ch_names[-1]] = 0
    
ch_keys = list(ch_dict.keys())
    
for i in range(0, n_epochs):
    single_epoch_data = encode_epochs_data[i, :, :]
    eeg_channels = single_epoch_data[0:128, :]
    bipolar_channel = single_epoch_data[-1, :]

    if np.sum((abs(eeg_channels) > 100e-6)) >= 1: # removing epochs with eeg channel amplitude greater than 100e-6
        rejected_epochs.append(i)
        
    elif np.sum((abs(bipolar_channel) > 200e-6)) >= 1: # removing epochs with bipolar channel amplitude greater than 200e-6
        rejected_epochs.append(i)
        ch_dict[ch_keys[-1]] += 1
        if i not in rejected_epochs: ## added 05/12/22
                rejected_epochs.append(i)
        
    for j in range(0, 128):
        single_channel_data = eeg_channels[j, :]
        if np.sum(abs(single_channel_data) > 100e-6) >= 1:
            ch_dict[ch_keys[j]] += 1
        
encoding_rejected = rejected_epochs

### Plot number of trials excluded

In [ ]:
## Remove channels with no epochs rejected ##
ch_dict_new = {}

for key in ch_dict.keys():
    if ch_dict[key] != 0:
        ch_dict_new[key] = ch_dict[key]

x = np.arange(0, len(ch_dict_new))
height = (np.array(list(ch_dict_new.values())) / n_epochs) * 100
tick_label = list(ch_dict_new.keys())

percent_rejected = (len(rejected_epochs) / n_epochs) * 100

plt.bar(x, height, color = "gray", alpha = 0.5)
plt.title(ID + " Encode Drop Log," + " Total Rejected = " + str(np.round(percent_rejected, 2)) + "%")
ax = plt.gca()
ax.set_xticks(x)
ax.set_xticklabels(tick_label)
ax.tick_params(axis = 'x', labelrotation = 45)
ax.set_ylabel("% Rejected")
plt.grid(axis = 'y')

### Drop Rejected Epochs 

In [ ]:
encode_epochs.drop(np.array(rejected_epochs))

## Apply Artifact Rejection To Baseline Period

In [ ]:
baseline_epochs_for_rejection = encode_epochs_copy.crop(encode_epochs_copy.tmin, 0)

In [ ]:
baseline_rejection_data = baseline_epochs_for_rejection.get_data()
n_epochs, n_channels, n_times = baseline_rejection_data.shape
rejected_epochs = []

ch_names = raw.ch_names
ch_dict = {}

for ch in ch_names[0:128]:
    ch_dict[ch] = 0

ch_dict[ch_names[-1]] = 0
    
ch_keys = list(ch_dict.keys())

for i in range(0, n_epochs):
    single_epoch_data = baseline_rejection_data[i, :, :]
    eeg_channels = single_epoch_data[0:128, :]
    bipolar_channel = single_epoch_data[-1, :]

    if np.sum((abs(eeg_channels) > 100e-6)) >= 1: # removing epochs with eeg channel amplitude greater than 100e-6
        rejected_epochs.append(i)
        
    elif np.sum((abs(bipolar_channel) > 200e-6)) >= 1: # removing epochs with bipolar channel amplitude greater than 200e-6
        ch_dict[ch_keys[-1]] += 1
        if i not in rejected_epochs:
            rejected_epochs.append(i)
        
    for j in range(0, 128):
        single_channel_data = eeg_channels[j, :]
        if np.sum(abs(single_channel_data) > 100e-6) >= 1:
            ch_dict[ch_keys[j]] += 1
        
baseline_rejected_epochs = rejected_epochs

### Plot number of trials excluded

In [ ]:
## Remove channels with no epochs rejected ##
ch_dict_new = {}

for key in ch_dict.keys():
    if ch_dict[key] != 0:
        ch_dict_new[key] = ch_dict[key]

x = np.arange(0, len(ch_dict_new))
height = (np.array(list(ch_dict_new.values())) / n_epochs) * 100
tick_label = list(ch_dict_new.keys())

percent_rejected = (len(rejected_epochs) / n_epochs) * 100

plt.bar(x, height, color = "gray", alpha = 0.5)
plt.title(ID + " Baseline Drop Log," + " Total Rejected = " + str(np.round(percent_rejected, 2)) + "%")
ax = plt.gca()
ax.set_xticks(x)
ax.set_xticklabels(tick_label)
ax.tick_params(axis = 'x', labelrotation = 45)
ax.set_ylabel("% Rejected")
plt.grid(axis = 'y')

## Create Reproduction Epochs
- Epochs are 1200ms duration, locked to the start of the reproduction epoch (indicated by cue?), using 100ms pre-stimulus period as baseline

In [ ]:
repo_event_dict = {'Reproduction/600':151,'Reproduction/650':152,'Reproduction/700':153,'Reproduction/750':154,
                   'Reproduction/800':155, 'Reproduction/850':156,'Reproduction/900':157,'Reproduction/950':158,
                   'Reproduction/1000':159}

repo_epochs = mne.Epochs(raw, events, event_id = repo_event_dict, preload = True)

### Add RT related Metadata to the Reproduction Epochs

In [ ]:
rt = events[repo_epochs.selection + 1] - events[repo_epochs.selection]
rt = rt[:,0] * 1000/512
mean_rt = np.zeros(len(rt))
median_rt = np.zeros(len(rt))

gaps = events[repo_epochs.selection]-events[repo_epochs.selection-1]
gaps=gaps[:,0]*1000/512

RT_gap = rt + gaps

for j in range(0,len(av_response_time)):
    mean_rt[repo_epochs.events[:,2]==151+j]=av_response_time[j]+50
    median_rt[repo_epochs.events[:,2]==151+j]=median_response_time[j]+50

Over_Under=np.zeros(len(rt))
Over_Under[rt>median_rt]=1
Trigger=repo_epochs.events[:,2]
repo_metadata={'trial_number':range(len(repo_epochs.events)),
         'Response_Time':rt,
         'Over_Under':Over_Under,
         'Mean_RT':mean_rt,
         'Median_RT':median_rt,
         'Trigger':Trigger,
         'Gap': gaps,
         'Reject_Artif': np.zeros(len(repo_epochs.events)),
         'RT_gap': RT_gap}
repo_metadata = pd.DataFrame(repo_metadata)

### Create Reproduction Epochs Object

In [ ]:
reproduction_events = repo_epochs.events 
event_id = repo_event_dict
preload = False
verbose = False 
detrend = None # 1 is linear detrend
metadata = repo_metadata 

if baseline_setting == 0: # not using this version currently
    tmin = -0.2 
    tmax = 1.2 
    baseline = (-0.2, 0) # baselined to the average of activity from -200ms to 0ms from start of epoch
    reject = None 

elif baseline_setting == 1:
    tmin = -0.3 # changed 16/12/22
    tmax = 0.9 # changed 16/12/22
    baseline = None 
    reject = None 
    
repo_epochs = mne.Epochs(raw, reproduction_events, event_id = repo_event_dict, tmin = tmin, tmax = tmax, 
                         reject = reject, preload = preload, verbose = False, detrend = detrend,
                         metadata = metadata, baseline = baseline)

repo_epochs.load_data()

### Baseline Epochs
Baselining reproduction epochs by subtracting pre-interval baseline period from all times in reproduction epoch

In [ ]:
if baseline_setting == 1:
    repo_epochs_data = repo_epochs.get_data()
    repo_epochs_data_baselined = np.zeros(repo_epochs_data.shape)
    
    for i in range(0, n_epochs):
        for j in range(0, n_channels):
            baseline = baseline_period[i, j]
            repo_epochs_data_baselined[i, j, :] = repo_epochs_data[i, j, :] - baseline

### Recreate Epochs Object
Have to do this again with new data if baselining to pre-interval period

Doing it before artifact rejection because I have to .drop the rejects epochs from an epochs object

In [ ]:
if baseline_setting == 1:
    
    data = repo_epochs_data_baselined
    reproduction_events = repo_epochs.events 
    event_id = repo_event_dict
    tmin = repo_epochs.tmin 
    reject = None
    baseline = None
    info = repo_epochs.info
    
    repo_epochs = mne.EpochsArray(data = data, info = info, events = reproduction_events, event_id = event_id, 
                                  tmin = tmin, reject = reject, baseline = baseline, metadata = metadata)

### Apply Artifact Rejection

In [ ]:
repo_epochs_data = repo_epochs.get_data()
n_epochs, n_channels, n_times = repo_epochs_data.shape
rejected_epochs = []

ch_names = raw.ch_names
ch_dict = {}

for ch in ch_names[0:128]:
    ch_dict[ch] = 0

ch_dict[ch_names[-1]] = 0
    
ch_keys = list(ch_dict.keys())

for i in range(0, n_epochs):
    single_epoch_data = repo_epochs_data[i, :, :]
    eeg_channels = single_epoch_data[0:128, :]
    bipolar_channel = single_epoch_data[-1, :]

    if np.sum((abs(eeg_channels) > 100e-6)) >= 1: # removing epochs with eeg channel amplitude greater than 100e-6
        rejected_epochs.append(i)
        
    elif np.sum((abs(bipolar_channel) > 200e-6)) >= 1: # removing epochs with bipolar channel amplitude greater than 200e-6
        ch_dict[ch_keys[-1]] += 1
        if i not in rejected_epochs:
            rejected_epochs.append(i)
        
    for j in range(0, 128):
        single_channel_data = eeg_channels[j, :]
        if np.sum(abs(single_channel_data) > 100e-6) >= 1:
            ch_dict[ch_keys[j]] += 1

repo_rejected_epochs = rejected_epochs

#### Plot number of trials excluded

In [ ]:
## Remove channels with no epochs rejected ##
ch_dict_new = {}

for key in ch_dict.keys():
    if ch_dict[key] != 0:
        ch_dict_new[key] = ch_dict[key]

x = np.arange(0, len(ch_dict_new))
height = (np.array(list(ch_dict_new.values())) / n_epochs) * 100
tick_label = list(ch_dict_new.keys())

percent_rejected = (len(rejected_epochs) / n_epochs) * 100

plt.bar(x, height, color = "gray", alpha = 0.5)
plt.title(ID + " Reproduction Drop Log," + " Total Rejected = " + str(np.round(percent_rejected, 2)) + "%")
ax = plt.gca()
ax.set_xticks(x)
ax.set_xticklabels(tick_label)
ax.tick_params(axis = 'x', labelrotation = 45)
ax.set_ylabel("% Rejected")
plt.grid(axis = 'y')

#### Add trial numbers of baseline epochs that were rejected 

In [ ]:
for i in baseline_rejected_epochs:
    if i not in rejected_epochs:
        rejected_epochs.append(i)

### Drop Rejected Epochs 

In [ ]:
repo_epochs.drop(np.array(rejected_epochs))

## Create Response Locked Epochs

In [ ]:
response_event_dict = {'Response/600':201,'Response/650':202,'Response/700':203,'Response/750':204,'Response/800':205,
              'Response/850':206,'Response/900':207,'Response/950':208,'Response/1000':209}

response_epochs = mne.Epochs(raw, events, event_id = response_event_dict, preload = True)

### Add RT related metadata

In [ ]:
from matplotlib import pyplot as plt
rt = events[response_epochs.selection-1] - events[response_epochs.selection]
rt=-rt[:,0]*1000/512
plt.hist(rt)
mean_rt=np.zeros(len(rt))
median_rt=np.zeros(len(rt))

gaps = events[response_epochs.selection-1]-events[response_epochs.selection-2]
gaps=gaps[:,0]*1000/512

RT_gap = rt + gaps

for j in range(0,len(av_response_time)):
    mean_rt[response_epochs.events[:,2]==201+j]=av_response_time[j]+50
    median_rt[response_epochs.events[:,2]==201+j]=median_response_time[j]+50

Over_Under=np.zeros(len(rt))
Over_Under[rt>=median_rt]=1
Trigger=response_epochs.events[:,2]
response_locked_metadata={'trial_number':range(len(response_epochs.events)),
         'Response_Time':rt,
         'Over_Under':Over_Under,
         'Mean_RT':mean_rt,
         'Median_RT':median_rt,
         'Trigger':Trigger,
         'Gap': gaps,
         'Reject_Artif': np.zeros(len(response_epochs.events)),
         'RT_gap': RT_gap}
response_locked_metadata=pd.DataFrame(response_locked_metadata)

### Create Response Locked Epochs Object
Creating epochs locked to the response in the response epoch

In [ ]:
response_locked_events = response_epochs.events 
event_id = response_event_dict 
metadata = response_locked_metadata 
detrend = None # 1 is linear detrend
preload = False
verbose = False

if baseline_setting == 0: # not using this version currently
    tmin = -1.2
    tmax = 0.2 
    reject = None
    baseline = (-1.2, -1.0) # epochs baselined to averaged of activity between -1200ms and -1000ms

if baseline_setting == 1:
    tmin = -1.1 # epochs are -1200ms from response
    tmax = 0.1 # epochs are 200ms from response
    reject = None
    baseline = None # will be baselined to pre-interval period after
    
response_locked_epochs = mne.Epochs(raw, events = response_locked_events, event_id = event_id, tmin = tmin, tmax = tmax,
                             reject = reject, metadata = metadata, preload = preload, detrend = detrend, verbose = verbose,
                             baseline = baseline)

response_locked_epochs.load_data()

### Baseline Epochs

In [ ]:
if baseline_setting == 1:
    response_locked_epochs_data = response_locked_epochs.get_data()
    response_locked_epochs_data_baselined = np.zeros(response_locked_epochs_data.shape)
    
    for i in range(0, n_epochs):
        for j in range(0, n_channels):
            baseline = baseline_period[i, j]
            response_locked_epochs_data_baselined[i, j, :] = response_locked_epochs_data[i, j, :] - baseline

### Recreate Epochs Object

In [ ]:
if baseline_setting == 1:
    
    data = response_locked_epochs_data_baselined
    response_locked_events = response_epochs.events 
    event_id = response_event_dict 
    tmin = response_locked_epochs.tmin # sets the times attribute to -1.2 start point rather than starting at 0
    reject = None
    baseline = None
    info = response_locked_epochs.info
    
    response_locked_epochs = mne.EpochsArray(data = data, info = info, events = response_locked_events, event_id = event_id, 
                                  tmin = tmin, reject = reject, baseline = baseline, metadata = metadata)

### Apply Artifact Rejection

In [ ]:
response_locked_epochs_data = response_locked_epochs.get_data()
n_epochs, n_channels, n_times = response_locked_epochs_data.shape
rejected_epochs = []

ch_names = raw.ch_names
ch_dict = {}

for ch in ch_names[0:128]:
    ch_dict[ch] = 0

ch_dict[ch_names[-1]] = 0
    
ch_keys = list(ch_dict.keys())

for i in range(0, n_epochs):
    single_epoch_data = response_locked_epochs_data[i, :, :]
    eeg_channels = single_epoch_data[0:128, :]
    bipolar_channel = single_epoch_data[-1, :]

    if np.sum((abs(eeg_channels) > 100e-6)) >= 1: # removing epochs with eeg channel amplitude greater than 100e-6
        rejected_epochs.append(i)
        
    elif np.sum((abs(bipolar_channel) > 200e-6)) >= 1: # removing epochs with bipolar channel amplitude greater than 200e-6
        ch_dict[ch_keys[-1]] += 1
        if i not in rejected_epochs:
            rejected_epochs.append(i)
        
    for j in range(0, 128):
        single_channel_data = eeg_channels[j, :]
        if np.sum(abs(single_channel_data) > 100e-6) >= 1:
            ch_dict[ch_keys[j]] += 1
    
response_locked_rejected_epochs = rejected_epochs

#### Plot number of trials excluded

In [ ]:
## Remove channels with no epochs rejected ##
ch_dict_new = {}

for key in ch_dict.keys():
    if ch_dict[key] != 0:
        ch_dict_new[key] = ch_dict[key]

x = np.arange(0, len(ch_dict_new))
height = (np.array(list(ch_dict_new.values())) / n_epochs) * 100
tick_label = list(ch_dict_new.keys())

percent_rejected = (len(rejected_epochs) / n_epochs) * 100

plt.bar(x, height, color = "gray", alpha = 0.5)
plt.title(ID + " Response Locked Drop Log," + " Total Rejected = " + str(np.round(percent_rejected, 2)) + "%")
ax = plt.gca()
ax.set_xticks(x)
ax.set_xticklabels(tick_label)
ax.tick_params(axis = 'x', labelrotation = 45)
ax.set_ylabel("% Rejected")
plt.grid(axis = 'y')

#### Add trial numbers of baseline epochs that were rejected 

In [ ]:
for i in baseline_rejected_epochs:
    if i not in rejected_epochs:
        rejected_epochs.append(i)

### Drop Rejected Epochs 

In [ ]:
response_locked_epochs.drop(np.array(rejected_epochs))

## Create Offset Epochs
- Go from end of encoding epoch onwards for a bit

In [ ]:
offset_event_dict = {'End_Encode/600':101,'End_Encode/650':102,'End_Encode/700':103,
                  'End_Encode/750':104,'End_Encode/800':105,'End_Encode/850':106,
                  'End_Encode/900':107,'End_Encode/950':108,'End_Encode/1000':109}

offset_epochs = mne.Epochs(raw, events, event_id = offset_event_dict, preload = True)

### Add RT related Metadata to the offset Epochs

In [ ]:
rt=events[offset_epochs.selection+2]-events[offset_epochs.selection+1]
rt=rt[:,0]*1000/512
mean_rt=np.zeros(len(rt))
median_rt=np.zeros(len(rt))

gaps = events[offset_epochs.selection+1]-events[offset_epochs.selection]
gaps=gaps[:,0]*1000/512

RT_gap = rt + gaps

for j in range(0,len(av_response_time)):
    mean_rt[offset_epochs.events[:,2]==101+j]=av_response_time[j]+50
    median_rt[offset_epochs.events[:,2]==101+j]=median_response_time[j]+50

Over_Under=np.zeros(len(rt))
Over_Under[rt>median_rt]=1
Trigger=offset_epochs.events[:,2]
offset_metadata={'trial_number':range(len(offset_epochs.events)),
         'Response_Time':rt,
         'Over_Under':Over_Under,
         'Mean_RT':mean_rt,
         'Median_RT':median_rt,
         'Trigger':Trigger,
         'Gap': gaps,
         'Reject_Artif': np.zeros(len(offset_epochs.events)),
         'RT_gap': RT_gap}

offset_metadata=pd.DataFrame(offset_metadata)

### Create Offset Epochs Object

In [ ]:
offset_epoch_events = offset_epochs.events 
event_id = offset_event_dict
preload = False
verbose = False 
metadata = offset_metadata 
detrend = None

tmin = -0.1 
tmax =  1 
reject = None
baseline = None 
   
offset_epochs = mne.Epochs(raw, offset_epoch_events, event_id = offset_event_dict, tmin = tmin, tmax = tmax, 
                           reject = reject, preload = preload, detrend = detrend, baseline = baseline, 
                           verbose = verbose, metadata = metadata)

offset_epochs.load_data()

### Baseline Epochs

In [ ]:
offset_epochs_data = offset_epochs.get_data()
offset_epochs_data_baselined = np.zeros(offset_epochs_data.shape)

for i in range(0, n_epochs):
    for j in range(0, n_channels):
        baseline = baseline_period[i, j]
        offset_epochs_data_baselined[i, j, :] = offset_epochs_data[i, j, :] - baseline

### Recreate Epochs Object
Have to do this again with new data if baselining to pre-interval period

In [ ]:
data = offset_epochs_data_baselined
offset_epoch_events = offset_epochs.events 
event_id = offset_event_dict
tmin = offset_epochs.tmin
reject = None
baseline = None
info = offset_epochs.info

offset_epochs = mne.EpochsArray(data = data, info = info, events = offset_epoch_events, event_id = event_id, 
                              tmin = tmin, reject = reject, baseline = baseline, metadata = metadata)

### Apply Artifact Rejection

In [ ]:
offset_epochs_data = offset_epochs.get_data()
n_epochs, n_channels, n_times = offset_epochs_data.shape
rejected_epochs = []

ch_names = raw.ch_names
ch_dict = {}

for ch in ch_names[0:128]:
    ch_dict[ch] = 0

ch_dict[ch_names[-1]] = 0
    
ch_keys = list(ch_dict.keys())

for i in range(0, n_epochs):
    single_epoch_data = offset_epochs_data[i, :, :]
    eeg_channels = single_epoch_data[0:128, :]
    bipolar_channel = single_epoch_data[-1, :]

    if np.sum((abs(eeg_channels) > 100e-6)) >= 1: # removing epochs with eeg channel amplitude greater than 100e-6
        rejected_epochs.append(i)
        
    elif np.sum((abs(bipolar_channel) > 200e-6)) >= 1: # removing epochs with bipolar channel amplitude greater than 200e-6
        ch_dict[ch_keys[-1]] += 1
        if i not in rejected_epochs:
            rejected_epochs.append(i)
        
    for j in range(0, 128):
        single_channel_data = eeg_channels[j, :]
        if np.sum(abs(single_channel_data) > 100e-6) >= 1:
            ch_dict[ch_keys[j]] += 1

offset_rejected_epochs = rejected_epochs

#### Plot number of trials excluded

In [ ]:
## Remove channels with no epochs rejected ##
ch_dict_new = {}

for key in ch_dict.keys():
    if ch_dict[key] != 0:
        ch_dict_new[key] = ch_dict[key]

x = np.arange(0, len(ch_dict_new))
height = (np.array(list(ch_dict_new.values())) / n_epochs) * 100
tick_label = list(ch_dict_new.keys())

percent_rejected = (len(rejected_epochs) / n_epochs) * 100

plt.bar(x, height, color = "gray", alpha = 0.5)
plt.title(ID + " Offset Drop Log," + " Total Rejected = " + str(np.round(percent_rejected, 2)) + "%")
ax = plt.gca()
ax.set_xticks(x)
ax.set_xticklabels(tick_label)
ax.tick_params(axis = 'x', labelrotation = 45)
ax.set_ylabel("% Rejected")
plt.grid(axis = 'y')

#### Add trial numbers of baseline epochs that were rejected 

In [ ]:
for i in baseline_rejected_epochs:
    if i not in rejected_epochs:
        rejected_epochs.append(i)

#### Drop Rejected Epochs 

In [ ]:
offset_epochs.drop(np.array(rejected_epochs))

# Update behavioural metadata to flag rejected trials

In [ ]:
encode_trials = encode_epochs.metadata["trial_number"]
repo_trials = repo_epochs.metadata["trial_number"]
response_locked_trials = response_locked_epochs.metadata["trial_number"]
offset_trials = offset_epochs.metadata["trial_number"]

In [ ]:
for i in range(0, len(encoding_metadata)):
    trial_num = encoding_metadata.iat[i, 0]
    if trial_num not in encode_trials:
        encoding_metadata.iat[i, 7] = 1
        
for i in range(0, len(repo_metadata)):
    trial_num = repo_metadata.iat[i, 0]
    if trial_num not in repo_trials:
        repo_metadata.iat[i, 7] = 1
        
for i in range(0, len(response_locked_metadata)):
    trial_num = response_locked_metadata.iat[i, 0]
    if trial_num not in response_locked_trials:
        response_locked_metadata.iat[i, 7] = 1
        
for i in range(0, len(offset_metadata)):
    trial_num = offset_metadata.iat[i, 0]
    if trial_num not in offset_trials:
        offset_metadata.iat[i, 7] = 1

## Save 

In [ ]:
fname = "Time_Reproduction_Data/" + "Alternate_Artifact_Rejection/" + baseline_name + "/Metadata/" + str(ID) + "_Encoding_Metadata_All.csv"
encoding_metadata.to_csv(fname, index = False)

fname = "Time_Reproduction_Data/" + "Alternate_Artifact_Rejection/" + baseline_name + "/Metadata/" + str(ID) + "_Repo_Metadata_All.csv"
repo_metadata.to_csv(fname, index = False)

fname = "Time_Reproduction_Data/" + "Alternate_Artifact_Rejection/" + baseline_name + "/Metadata/" + str(ID) + "_RL_Metadata_All.csv"
response_locked_metadata.to_csv(fname, index = False)

fname = "Time_Reproduction_Data/" + "Alternate_Artifact_Rejection/" + baseline_name + "/Metadata/" + str(ID) + "_Offset_Metadata_All.csv"
offset_metadata.to_csv(fname, index = False)

# Save Epochs for Doing CSD Transformation in MATLAB

## Encoding Epochs

In [ ]:
## Saving epochs in mat file form ##
import scipy.io
fname = "Time_Reproduction_Data/" + "Alternate_Artifact_Rejection/" + baseline_name + "/Epochs_Arrays/" + "Encode/" + str(ID) + "_Encode_All.mat"
encode_epochs_array = encode_epochs.get_data()
encode_epochs_array = np.moveaxis(encode_epochs_array, 0, -1)
mat_dict = {"encode_epochs_array": encode_epochs_array}
scipy.io.savemat(fname, mat_dict)

## Also want to save the non-csd epochs for the metadata ##
fname = "Time_Reproduction_Data/" + "Alternate_Artifact_Rejection/" + baseline_name + "/Epochs/" + "Encode/" + str(ID) + "_Encode_All_epo.fif"
encode_epochs.save(fname = fname, overwrite = True)

## Reproduction Epochs

In [ ]:
## Saving epochs in mat file form ##
fname = "Time_Reproduction_Data/" + "Alternate_Artifact_Rejection/" + baseline_name + "/Epochs_Arrays/" + "Reproduction/" + str(ID) + "_Reproduction_All.mat"
repo_epochs_array = repo_epochs.get_data()
repo_epochs_array = np.moveaxis(repo_epochs_array, 0, -1)
mat_dict = {"repo_epochs_array": repo_epochs_array}
scipy.io.savemat(fname, mat_dict)

## Also want to save non-csd epochs for metadata ##
fname = "Time_Reproduction_Data/" + "Alternate_Artifact_Rejection/" + baseline_name + "/Epochs/" + "Reproduction/" + str(ID) + "_Reproduction_All_epo.fif"
repo_epochs.save(fname = fname, overwrite = True)

## Response-Locked Epochs

In [ ]:
## Saving epochs in mat file form ##
fname = "Time_Reproduction_Data/" + "Alternate_Artifact_Rejection/" + baseline_name + "/Epochs_Arrays/" + "Response_Locked/" + str(ID) + "_Response_Locked_All.mat"
response_locked_epochs_array = response_locked_epochs.get_data()
response_locked_epochs_array = np.moveaxis(response_locked_epochs_array, 0, -1)
mat_dict = {"response_locked_epochs_array": response_locked_epochs_array}
scipy.io.savemat(fname, mat_dict)

## Also want to save non-csd epochs for metadata ##
fname = "Time_Reproduction_Data/" + "Alternate_Artifact_Rejection/" + baseline_name + "/Epochs/" + "Response_Locked/" + str(ID) + "_RL_All_epo.fif"
response_locked_epochs.save(fname = fname, overwrite = True)

## Offset Epochs

In [ ]:
from scipy.io import savemat

In [ ]:
## Saving epochs in mat file form ##
fname = "Time_Reproduction_Data/" + "Alternate_Artifact_Rejection/" + baseline_name + "/Epochs_Arrays/" + "Offset/" + str(ID) + "_Offset_All.mat"
offset_epochs_array = offset_epochs.get_data()
offset_epochs_array = np.moveaxis(offset_epochs_array, 0, -1)
mat_dict = {"offset_epochs_array": offset_epochs_array}
savemat(fname, mat_dict)

## Also want to save non-csd epochs for metadata ##
fname = "Time_Reproduction_Data/" + "Alternate_Artifact_Rejection/" + baseline_name + "/Epochs/" + "Offset/" + str(ID) + "_Offset_All_epo.fif"
offset_epochs.save(fname = fname, overwrite = True)